In [2]:
# installing dependencies
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 4.2 MB/s 
     |████████████████████████████████| 7.6 MB 48.9 MB/s 
     |████████████████████████████████| 163 kB 47.8 MB/s 


In [4]:
from transformers import pipeline
from bs4 import BeautifulSoup
import requests

In [5]:
# importing summarizer pipeline
summarizer = pipeline('summarization')

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


Downloading:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

In [19]:
URL = "https://hackernoon.com/will-the-game-stop-with-gamestop-or-is-this-just-the-beginning-2j1x32aa"

In [20]:
r = requests.get( URL)

In [21]:
soup = BeautifulSoup(r.text, 'html.parser')
results = soup.find_all(['h1','p'])


In [22]:
# Concatinating text to one block
text = [result.text for result in results]
article = ' '.join(text)

In [23]:
article

'Will The Game Stop with Gamestop Or Is This Just The Beginning? Crypto, Markets, Trading The GameStop squeeze on short-sellers is an extraordinary event in markets, where at face value, retail traders and investors have worked together in an attempt to put some of the largest wall street institutions out of business. The events can be interpreted with many viable lenses and there are ironies baked in that are pure serendipity. There has been a centrally controlled game in the global financial system in which insiders benefited while outsiders got hurt that comes to a head with a company called GameStop. The broking firm of most of the retail side of this warfare ‘RobinHood’ is literally stealing from its poor, retail investors to give to its rich, capital backers. One of the historical realities of this game has been that macro-investing – the sages of not only portfolio management, but often also sophisticated social and cultural figures – have had a hard time making money in markets

In [24]:
# Chunking text
article = article.replace('.', '<eos>')
article = article.replace('!', '!<eos>')
article = article.replace('?', '?<eos>')
sentences = article.split('<eos>')

In [27]:
sentences[4]

' The broking firm of most of the retail side of this warfare ‘RobinHood’ is literally stealing from its poor, retail investors to give to its rich, capital backers'

In [30]:
max_chunk = 500
current_chunk = 0
chunks = []

for sentence in sentences:
  if len(chunks) == current_chunk+1:
    if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
      chunks[current_chunk].extend(sentence.split(' '))
    else:
      current_chunk += 1
      chunks.append(sentence.split(' '))
  else:
    print(current_chunk)
    chunks.append(sentence.split(' '))





0


In [31]:
for chunk_id in range(len(chunks)):
  chunks[chunk_id] = ' '.join(chunks[chunk_id])

In [35]:
# Summarizing text
output = summarizer(chunks, max_length=115, min_length=90, do_sample=False)

In [36]:
' '.join([summ['summary_text'] for summ in output])

' The GameStop squeeze on short-sellers is an extraordinary event in markets, where at face value, retail traders and investors have worked together in an attempt to put some of the largest wall street institutions out of business . The broking firm of most of the retail side of this warfare ‘RobinHood’ is stealing from its poor, retail investors to give to its rich, capital backers . With government policy dictating markets in greater degrees since the mid-90’s, the politically connected have been more likely to survive while the sophisticated and  Victor Niederhoffer blew his hedge fund up in 1997 in a highly statistically improbable event, in which he sold puts that were targeted by market mechanics, rather than ‘truth’ The new type of market - which I call ‘the second bite of the apple’ - is a disadvantage to people who understand markets and understand risk . The fundamentals no longer matter; and this was demonstrated only 5 years later with what could be the largest bubble of vi

In [37]:
# Saving our file
text = ' '.join([summ['summary_text'] for summ in output])

with open('summaryblog.txt', 'w') as f:
  f.write(text)